# Stock AI Predictor - Google Colab Runner

This notebook allows you to run Stock AI Predictor models on Google Colab with minimal setup. It handles:
- Setting up the environment
- Connecting to Google Drive
- Transferring database files
- Running parameter testing and pattern mining scripts

**Date:** May 15, 2025

## Instructions

1. Upload your `data.db` file to your Google Drive
2. Run this notebook cell by cell
3. Adjust the path to your database file in the database setup section
4. Make sure all dependencies are installed correctly
5. Run the desired model scripts

## Database Path Configuration

The `setup_db_path` function ensures we can properly configure the database path dynamically for both local and Colab environments. This is crucial for maintaining code compatibility across different execution contexts.

In [ ]:
# Function to set up database path dynamically for both local and Colab environments
def setup_db_path():
    """
    Configure database path based on environment
    
    This function detects whether we're running in Colab and sets up the correct
    database path accordingly. It should be used in any script that needs to
    access the database.
    
    Returns:
        str: The correct path to the database file
    """
    import os
    
    # Check if we're running in Colab
    in_colab = 'google.colab' in str(get_ipython())
    
    if in_colab:
        # When in Colab, use the standard path
        db_path = "./Data/Storage/data.db"
    else:
        # When running locally, use the regular path
        # Get the current working directory
        cwd = os.getcwd()
        
        # Check if we're in a subdirectory of the project
        if os.path.basename(cwd) == 'Colab':
            db_path = "../Data/Storage/data.db"
        else:
            db_path = "./Data/Storage/data.db"
    
    # Ensure the directory exists
    os.makedirs(os.path.dirname(db_path), exist_ok=True)
    
    return db_path

## 1. Environment Setup

In [ ]:
# Install PyDrive for Google Drive integration
!pip install -q pydrive2

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
print("Google Drive mounted successfully!")

## 2. Repository Setup

There are two options to get the code:
1. Clone directly from GitHub
2. Upload from your Drive if you already have it there

Choose the most appropriate option for your workflow.

In [ ]:
# Option 1: Clone from GitHub
# Uncomment and modify with your own repository URL if needed
# !git clone https://github.com/yourusername/Stock_AI_Predictor.git
# %cd Stock_AI_Predictor

# Option 2: Copy from Google Drive (if you have it there)
import os
import shutil

# Define source path in your Google Drive
drive_repo_path = "/content/drive/MyDrive/Stock_AI_Predictor"  # Update this path
local_repo_path = "/content/Stock_AI_Predictor"

# Create directory for local repository
os.makedirs(local_repo_path, exist_ok=True)

# Function to copy directory contents recursively
def copy_directory(src, dst):
    for item in os.listdir(src):
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        if os.path.isdir(s):
            os.makedirs(d, exist_ok=True)
            copy_directory(s, d)
        else:
            shutil.copy2(s, d)

# Copy repository from Drive to Colab if it exists
if os.path.exists(drive_repo_path):
    copy_directory(drive_repo_path, local_repo_path)
    print(f"Repository copied from Google Drive to {local_repo_path}")
else:
    print(f"Repository not found at {drive_repo_path}. Please check the path or use git clone.")

# Change to repository directory
%cd /content/Stock_AI_Predictor

## 3. Install Dependencies

Install all required packages for Stock AI Predictor

In [ ]:
# Install all required dependencies
!pip install -r Colab/requirements.txt

# Verify key packages are installed
import pkg_resources
import pandas as pd

key_packages = ['numpy', 'pandas', 'scikit-learn', 'tensorflow', 'torch', 'yfinance', 'sqlitecloud']
for package in key_packages:
    try:
        version = pkg_resources.get_distribution(package).version
        print(f"{package}: {version}")
    except pkg_resources.DistributionNotFound:
        print(f"{package}: Not installed")

## 4. Database Setup

Copy and set up the database file from Google Drive

In [ ]:
import os
import shutil

# Define paths (adjust these according to your Drive structure)
drive_db_path = "/content/drive/MyDrive/Stock_AI_Predictor/data.db"  # Update this path
local_db_dir = "./Data/Storage/"
local_db_path = os.path.join(local_db_dir, "data.db")

# Create directory structure if it doesn't exist
os.makedirs(local_db_dir, exist_ok=True)
os.makedirs("./Data/Raw/Stocks", exist_ok=True)
os.makedirs("./Data/Raw/Sentiment", exist_ok=True)

# Copy database file
if os.path.exists(drive_db_path):
    shutil.copy(drive_db_path, local_db_path)
    print(f"Database copied from {drive_db_path} to {local_db_path}")
    
    # Verify database file
    if os.path.exists(local_db_path):
        print(f"Database size: {os.path.getsize(local_db_path) / (1024*1024):.2f} MB")
    else:
        print("Database copy failed. Check file permissions.")
else:
    print(f"Warning: Database file not found at {drive_db_path}")
    print("You'll need to create or download a new database.")

## Database Path Patching

This section applies necessary patches to ensure the database connections work correctly in the Colab environment.

In [ ]:
# Run the database patcher to ensure compatibility
from Colab.colab_db_helper import setup_database
from Colab.colab_db_patcher import patch_all_db_files

# Define path to your database in Google Drive
drive_db_path = "/content/drive/MyDrive/Stock_AI_Predictor/data.db"  # Update this path

# Setup the database (copy from Drive to local storage)
db_setup_success = setup_database(drive_db_path)

# Patch database-related files if setup was successful
if db_setup_success:
    patched_files = patch_all_db_files()
    print(f"Successfully patched {patched_files} database-related files")
else:
    print("Database setup failed. Check the paths and permissions.")

## 5. Path Configuration

Set up Python path to include the project modules

In [ ]:
import sys
import os

# Add current directory to path so Python can find the modules
sys.path.append('.')

# Test importing modules to verify project structure
try:
    from Core import engine_v2
    print("Core module imported successfully!")
except ImportError as e:
    print(f"Error importing project modules: {e}")
    print("Check that the project structure is correct.")

# Verify important directories exist
important_dirs = ['Core', 'Data', 'Colab', 'Pattern', 'RL', 'Sentiment']
for dir_name in important_dirs:
    if os.path.isdir(dir_name):
        print(f"✓ {dir_name} directory found")
    else:
        print(f"✗ {dir_name} directory NOT found")

## 6. Database Connection Test

Verify that we can connect to the database

In [ ]:
import sqlite3
import pandas as pd

# Test database connection
try:
    conn = sqlite3.connect(local_db_path)
    # Get list of tables
    tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
    print("Database tables:")
    for table in tables['name']:
        print(f"- {table}")
    conn.close()
    print("\nDatabase connection successful!")
except Exception as e:
    print(f"Database connection error: {e}")

## Run Models

Now that the environment is set up, you can run the various models and scripts. Uncomment the cells below to run specific models.

In [ ]:
# Run parameter tester (multi-threaded version)
try:
    print("Running multi-threaded parameter tester...")
    %run Colab/parameter_tester_multithreaded.py
    print("Parameter testing completed successfully")
except Exception as e:
    print(f"Error running parameter tester: {e}")

In [ ]:
# Run pattern miner (uncomment to run)
'''
try:
    print("Running PIP pattern miner...")
    %run Colab/pip_pattern_miner.py
    print("Pattern mining completed successfully")
except Exception as e:
    print(f"Error running pattern miner: {e}")
'''

In [ ]:
# Run standard parameter tester (uncomment to run)
'''
try:
    print("Running standard parameter tester...")
    %run Colab/parameter_tester.py
    print("Standard parameter testing completed successfully")
except Exception as e:
    print(f"Error running standard parameter tester: {e}")
'''

## Save Results

After running the models, you can save the results back to Google Drive.

In [ ]:
# Save results back to Google Drive
import os
import shutil
from datetime import datetime

# Function to copy directory recursively
def copy_directory(src, dst):
    if not os.path.exists(dst):
        os.makedirs(dst)
    for item in os.listdir(src):
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        if os.path.isdir(s):
            copy_directory(s, d)
        else:
            if not os.path.exists(os.path.dirname(d)):
                os.makedirs(os.path.dirname(d))
            shutil.copy2(s, d)

# Create a timestamped directory for results
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
drive_results_dir = f"/content/drive/MyDrive/Stock_AI_Predictor/Results_{timestamp}"
os.makedirs(drive_results_dir, exist_ok=True)

# Define directories to save
dirs_to_save = [
    "Images",           # Save generated images
    "Data/Storage",     # Save updated database
    "Experements"       # Save experiment results
]

# Copy each directory to Google Drive
for dir_path in dirs_to_save:
    src_path = os.path.join(".", dir_path)
    dst_path = os.path.join(drive_results_dir, dir_path)
    
    if os.path.exists(src_path):
        # Copy directory
        try:
            copy_directory(src_path, dst_path)
            print(f"Copied {src_path} to Google Drive")
        except Exception as e:
            print(f"Error copying {src_path}: {e}")
    else:
        print(f"Directory not found: {src_path}")

print(f"\nAll results saved to Google Drive at: {drive_results_dir}")